In [331]:
import pandas as pd
import datetime
import statsmodels.formula.api as sm
import statsmodels.api as smap
import matplotlib.pyplot as plt


In [193]:
# Code cleans up data for prices (monthly, state)

data = pd.ExcelFile('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/monthly_prices_sales_revenue.xlsx')

df = data.parse(sheetname=data.sheet_names[0], skiprows=0)

# make a list of the header row and strip up to the 4th letter. This is the location and year information
cols1 = list(df.columns)
cols1 = [str(x).lower() for x in cols1]

# make another list of the first row,this is the age group information
# we need to preserve this information in the column name when we reshape the data 
cols2 = list(df.iloc[0,:])
cols2 = [str(x).lower() for x in cols2]

cols3 = list(df.iloc[1,:])
cols3 = [str(x).lower().replace(" ","_") for x in cols3]

# now join the two lists to make a combined column name which preserves our location, year and age-group information
cols = [x+"_"+y for x,y in zip(cols1,cols2)]
name = ""
for i,x in enumerate(cols):
    if "unnamed" in x:
        x = x.split('_')[-1]
        cols[i] = x
    if "_" in x:
        name = x.split("_")[0]
        continue
    elif ("_" not in x):
        cols[i] = name+"_" + x
cols[0:4] = cols3[0:4]
# Assign new column names to the dataframe
df.columns = cols
df.drop([0,1], inplace=True)
df = df[:-1]
df.to_csv('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/avg_price_monthly.csv')

In [442]:
# Import electricity price data (annual)

price_data = pd.read_csv('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/avg_price_monthly.csv')
price_data['date'] = pd.to_datetime([f'{y}-{m}-01' for y, m in zip(price_data.year, price_data.month)])

# Import generator data (survey of all generators in 2018)

gen_data = pd.read_csv('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/eia_data/eia8602018/merged_new.csv')
gen_data = gen_data.drop(gen_data.columns[gen_data.columns.str.contains("Unnamed")], axis=1)
gen_data['operating_year'] = gen_data['operating_year'].astype(int)
gen_data['operating_month'] = gen_data['operating_month'].astype(int)
gen_data['operating_date'] = pd.to_datetime([f'{y}-{m}-01' for y, m in zip(gen_data.operating_year, gen_data.operating_month)])
gen_data['summer_cap']=pd.to_numeric(gen_data['summer_capacity_mw'].str.replace(",","").replace(" ",""))
gen_data['winter_cap']=pd.to_numeric(gen_data['winter_capacity_mw'].str.replace(",","").replace(" ",""))

/Users/rachelanderson/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (18,41,45,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [443]:
solar = gen_data[gen_data['technology'] == "Solar Photovoltaic"]
solar = solar[solar['grant_program']=='none']

In [457]:
solar_cap_monthly = solar.groupby(['plant_state','operating_date','operating_month','operating_year'])[['summer_cap','nameplate_cap', 'winter_cap']].sum().reset_index()
res_price_monthly = price_data[['date', 'state', 'residential_price']]

In [460]:
monthly_vars = solar_cap_monthly.merge(res_price_monthly, how='right', left_on = ['operating_date', 'plant_state'], right_on=['date','state'])
monthly_vars = monthly_vars.drop(columns=['plant_state','operating_date'], axis = 1)
monthly_vars=monthly_vars.fillna(0)

In [461]:
monthly_vars.to_csv('/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/monthly_data.csv')

In [434]:
grant_data = pd.read_excel('../../../Data for Tax Equity Project/treasury_data/grant_awards.xlsx',header=1)

In [463]:
grant_data.groupby('State').Funded.sum().reset_index()

,State,Funded
0,AK,2.587784e+07
1,AL,2.495648e+06
2,AR,2.575290e+05
3,AZ,1.444632e+09
4,CA,7.045618e+09
5,CO,5.454013e+08
6,CT,1.414500e+08
7,DC,3.525566e+06
8,DE,1.021639e+08
9,FL,3.813748e+08
